In [2]:
# os.chdir ..
import os
os.chdir("..")

print (os.getcwd())

/home/colligo/project/vlm/nanoGPT


In [3]:
pwd

'/home/colligo/project/vlm/nanoGPT'

# 1 - Basics for flex attention

https://pytorch.org/blog/flexattention/

In [ ]:
import torch

def get_dummy_qkv(B, T, n_embd, n_head, device="cpu"):
    q = torch.randn(B, n_head, T, n_embd // n_head)
    k = torch.randn(B, n_head, T, n_embd // n_head)
    v = torch.randn(B, n_head, T, n_embd // n_head)
    # also put them on device
    q = q.to(device)
    k = k.to(device)
    v = v.to(device)
    return q, k, v

B, T, n_embd, n_head = 1, 10, 768, 12
q, k, v = get_dummy_qkv(B, T, n_embd, n_head, device="cuda")

attn_out = flex_attention(q, k, v, block_mask=block_mask)
print (attn_out.shape)

In [11]:
B, T, n_embd, n_head = 1, 10, 768, 12
q, k, v = get_dummy_qkv(B, T, n_embd, n_head)

print (q.shape, k.shape, v.shape)

torch.Size([1, 12, 10, 64]) torch.Size([1, 12, 10, 64]) torch.Size([1, 12, 10, 64])


## 1.1 - score_mod

In [9]:
def noop(score, b, h, q_idx, kv_idx):
    return score

from torch.nn.attention.flex_attention import flex_attention

attn_out = flex_attention(q, k, v, score_mod=noop)
print (attn_out.shape)

torch.Size([1, 12, 10, 64])


In [10]:
def causal_mask(score, b, h, q_idx, kv_idx):
    return torch.where(q_idx >= kv_idx, score, -float("inf"))

attn_out = flex_attention(q, k, v, score_mod=causal_mask)
print (attn_out.shape)

torch.Size([1, 12, 10, 64])


## 1.2 - Mask Mod

While most causal or other mask mods can be implemented using score mod (e.g. by setting value to -inf), it does not leverage sparsity of the mask matrix so is slower. mask_mod better leverages the sparsity of the flex attn mask


>However, masking is special compared to other modifications – if something is masked out, we can completely skip its computation! In this case, a causal mask has about 50% sparsity, so not taking advantage of the sparsity would result in a 2x slowdown. Although this score_mod is sufficient to implement causal masking correctly, getting the performance benefits of sparsity requires another concept – mask_mod.



In [13]:
from torch.nn.attention.flex_attention import create_block_mask

def causal(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx

block_mask = create_block_mask(causal, B=None, H=None, Q_LEN=10, KV_LEN=10)

attn_out = flex_attention(q, k, v, block_mask=block_mask)
print (attn_out.shape)


RuntimeError: Expect q/k/v and block_mask to be on the same device but got cpu and cuda:0.

In [15]:
block_mask

BlockMask(
    kv_num_blocks=torch.Size([1, 1, 1]),
    kv_indices=torch.Size([1, 1, 1, 1]),
    full_kv_num_blocks=torch.Size([1, 1, 1]),
    full_kv_indices=torch.Size([1, 1, 1, 1]),
    q_num_blocks=torch.Size([1, 1, 1]),
    q_indices=torch.Size([1, 1, 1, 1]),
    full_q_num_blocks=torch.Size([1, 1, 1]),
    full_q_indices=torch.Size([1, 1, 1, 1]),
    BLOCK_SIZE=(128, 128),
    shape=(1, 1, 10, 10),
    sparsity=-16284.00%,
    mask_mod=causal
)

# 2 - Flex attn for gpt2

# Debug